## Prepare the virtual environment 

In [ ]:
# !which python # using python2 from virtualenv
# !pip install pandas
# !pip install datetime
# !pip install scipy
# !pip install NumPy
# !pip install Matplotlib
# !pip install scikit-learn

## Import and loading in data

In [247]:
import json
import time
import copy
import math
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsRegressor


# importing the csv data about the store sales, 906 stores with 11936 timestamps.
raw_csv = pd.read_csv('sales_granular.csv', index_col=0)

# importing the surroundings information about some 546 stores, each with 89 possible types of surroundings.
raw = json.load(open('Surroundings.json'))

# manual inspection of JSON file by writing a single item in the list to disk.
with open('extract.json', 'w') as outfile:
    json.dump(raw[0], outfile)


## Helper functions

In [248]:
def group_by_month(timestamp_str):
    """
    Groups the sparse timestamps of each store inside a monthly groupe. 
    This is relevant since the number of timestamps in not consistent across periods in a month.
    Furthermore more sophisticated models such as AMRA / ARIMA modeling require lag times of weeks or months.
    
    :param: timestam_str -> a string representing the hourly timestamp.
    :return: month_key -> the year-month timestamps which can be used for time-series analysis.
    """
    space_idx = timestamp_str.find(' ')
    parse_date_string = timestamp_str[:space_idx]
    
    date_datetime = datetime.strptime(parse_date_string, '%m/%d/%y')

    if len(str(date_datetime.month)) == 1:
        mm = '0' + str(date_datetime.month)
    else:
        mm = str(date_datetime.month)
        
    yyyy = str(date_datetime.year)
    
    month_key = '{0}-{1}'.format(yyyy, mm)
    return month_key

In [511]:
def get_series_stats(series, store_id):
    """
    Generates a dictionary of relevant statistics for a given time-series -like dataset.
    
    :param: series -> the monthly sales of a store.
    :param: store_id -> the unique identifier of a store.
    :return: dict -> a dictionary containing statistics about each store, identifiable by the store_id key.
    """
    sales_points_sum = 0
    sales_points_valid = []

    for _, val in enumerate(series):
        if math.isnan(val):
            continue
        else:
            sales_points_sum += val
            sales_points_valid.append(val)

    series_mean = float(sales_points_sum) / max(len(sales_points_valid), 1)
    series_stdev = max(round(math.sqrt(float(reduce(lambda x, y: x + y, map(lambda x: (x - series_mean) ** 2, sales_points_valid))) / len(sales_points_valid)), 2), 1)
    
    return dict({
        'mean': round(series_mean, 2),
        'months_of_data_count': len(sales_points_valid),
        'store_id': store_id,
        'total_products_sold': sales_points_sum,
        'stdev': series_stdev
        })
    

In [ ]:
def normalize_df(df):
    """
    Creates a normalized version of the dataframe used in training and testing.
    
    :param: df -> a pandas dataframe which is not normalized.
    :return: norml -> a pandas dataframe with normalized values between (0,1).
    """
    normalized_df = copy.deepcopy(df)
    norm1 = normalize(normalized_df, axis=0, norm='max')
    return(norm1)

In [527]:
# borrowed shamelessly from https://plot.ly/python/polygon-area/
def PolygonSort(corners):
    n = len(corners)
    cx = float(sum(x for x, y in corners)) / n
    cy = float(sum(y for x, y in corners)) / n
    cornersWithAngles = []
    for x, y in corners:
        an = (np.arctan2(y - cy, x - cx) + 2.0 * np.pi) % (2.0 * np.pi)
        cornersWithAngles.append((x, y, an))
    cornersWithAngles.sort(key = lambda tup: tup[2])
    return map(lambda (x, y, an): (x, y), cornersWithAngles)

def PolygonArea(corners):
    n = len(corners)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

## example of area calculation below | as reference: https://www.mathsisfun.com/geometry/polygons-interactive.html
# plotting the points in this order: N, E, S, W
corners = [(2.2, 5.4), (5.4, 4.4), (4.7, 1.7), (1.4, 2.5)]
corners_sorted = PolygonSort(corners)
area = PolygonArea(corners_sorted)

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

def plot_monthly_series(monthly_series):
    """
    Creates the scatter plot of a time series for easy of exploration.
    """
    plt.plot(monthly_series)
    plt.show()

## Data Transformation

In [514]:
# aggregated all the sales data per month based on the column names.
# sorted key list now contains indices per corresponsing month, used later for plotting and regressions.
atomic_timestamps = list(raw_csv)

month_dict = {}

for idx, col_timestamp in enumerate(atomic_timestamps):
    current_month = group_by_month(col_timestamp)
    
    if month_dict.has_key(current_month):
        month_dict[current_month].append(idx)
    else:
        month_dict[current_month] = []
        month_dict[current_month].append(idx)

sorted_key_list = sorted(month_dict.keys())

store_id_list = list(raw_csv.index)


# generate a dictionary of store_id with their appropriate series values (interpretable by month)
store_dict = {}

for i in range(0, len(raw_csv.index)):
    monthly_series = []
    for _, key in enumerate(sorted_key_list):
        monthly_series.append(raw_csv.iloc[i][month_dict[key][0]:month_dict[key][-1]].sum(skipna=True))
        
    store_dict[store_id_list[i]] = monthly_series


## Create table of relevant statistics in a time-series manner

In [615]:
stats_df = pd.DataFrame(columns= ['store_id', 'total_products_sold', 'mean', 'stdev', 'months_of_data_count'])

for key in store_dict.keys():
    row_dict = get_series_stats(store_dict[key], key) 
    row_df = pd.DataFrame.from_records(row_dict, index=[0])[['store_id', 'total_products_sold', 'mean', 'stdev', 'months_of_data_count']]
    stats_df = pd.concat([stats_df, row_df])

stats_df.reset_index(drop=True, inplace=True)

result = stats_df.sort_values(['months_of_data_count', 'total_products_sold', 'stdev'], ascending=[0, 0, 1])

# preparing the surroundings data for analysis
amenities_array = raw[0]['surroundings'].keys()

column_names_amenities = copy.deepcopy(amenities_array)
column_names_amenities = ['store_id'] + column_names_amenities

full_feature_amenities_df = pd.DataFrame(columns = column_names_amenities, index=[0])

for _, surroundings_obj in enumerate(raw):
    
    amenities_feature_dict = {}
    store_id = surroundings_obj['store_code']
    amenities_feature_dict['store_id'] = store_id
    
    for _, key in enumerate(amenities_array):
        amenities_feature_dict[key] = len(surroundings_obj['surroundings'][key]) 
        
    feature_amenities_row = pd.DataFrame(data = amenities_feature_dict, columns = column_names_amenities, index=[0])
    
    full_feature_amenities_df = pd.concat([full_feature_amenities_df, feature_amenities_row])

full_feature_amenities_df = full_feature_amenities_df[1:]
full_feature_amenities_df.reset_index(drop=True, inplace=True)

## Preparing fitting and validation data sets

In [618]:
# select the store ids that are only in the surrounding dataset
ops_df = result.loc[result['store_id'].isin(full_feature_amenities_df['store_id'])]

# join on store_id key to append the total_products_sold
merged_df = pd.merge(full_feature_amenities_df, ops_df[['store_id', 'total_products_sold']], on='store_id')

# make sure no NA rows are present (same dataset with or without the drop, but it's good practice)
merged_df.dropna()

,store_id,subway_station,department_store,embassy,beauty_salon,police,courthouse,cemetery,pharmacy,local_government_office,...,zoo,train_station,jewelry_store,laundry,insurance_agency,plumber,pet_store,bakery,travel_agency,mean
0,10055,0,0,0,4,0,0,0,3,0,...,0,0,0,1,2,0,0,3,1,2252.00
1,10077,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,557.14
2,10079,0,1,0,4,0,1,0,3,1,...,0,0,4,2,2,0,0,3,2,18014.00
3,10086,0,0,0,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,5635.00
4,10111,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,3000.00
5,10377,0,2,1,14,0,0,1,5,3,...,0,0,50,0,0,0,0,12,14,6436.36
6,10441,0,1,0,16,0,0,0,5,6,...,0,1,6,2,3,0,1,6,9,11470.00
7,10545,0,0,0,7,1,0,0,4,1,...,0,1,2,0,2,0,0,4,3,2535.00
8,10548,0,0,0,7,0,0,0,3,1,...,0,0,3,0,3,0,0,5,3,6406.88
9,10672,0,0,0,0,0,0,0,1,2,...,0,1,0,0,1,0,0,4,0,4341.43


## Start of cross validation implementation

In [619]:
# creating odd list of K for KNN
neigh_list = list(range(5,50))

# subsetting just the odd ones
neighbors = filter(lambda x: x % 2 != 0, neigh_list)

for _, weight_type in enumerate(['uniform', 'distance']):
    
    print('\n #### Starting with {} weight #### \n'.format(weight_type))
    
    for kn in neighbors:

        cross_validation_array = []

        for cv_iteration in range(0, 10):

            # sample random index numbers for splitting the dataset. Will go for 85% training and 15% testing.
            msk = np.random.rand(len(merged_df)) <= 0.80

            train_df = merged_df[msk]

            test_df = merged_df[~msk]

            training_fit_set_columns = [col for col in train_df.columns if col not in ['store_id', 'total_products_sold']]
            training_fit_df = normalize_df(train_df[training_fit_set_columns])
            training_target_df = train_df['total_products_sold']

            test_fit_set_columns = [col for col in test_df.columns if col not in ['store_id', 'total_products_sold']]
            test_fit_df = normalize_df(test_df[test_fit_set_columns])
            test_target_df = test_df['total_products_sold']

            # fit model 
            neigh = KNeighborsRegressor(n_neighbors=kn, weights=weight_type)
            neigh.fit(training_fit_df, training_target_df) 
            neigh.predict(test_fit_df)

            cross_validation_array.append(neigh.score(test_fit_df, test_target_df))


        avg_score = reduce(lambda x, y: x + y, cross_validation_array) / len(cross_validation_array)

        print('Average score for k equals {0} is {1}'.format(kn, avg_score))


 #### Starting with uniform weight #### 

Average score for k equals 5 is -0.194352420766
Average score for k equals 7 is -0.138053833365
Average score for k equals 9 is -0.0252118615761
Average score for k equals 11 is -0.0759648269458
Average score for k equals 13 is -0.0613749297847
Average score for k equals 15 is 0.0662578735688
Average score for k equals 17 is -0.10738487293
Average score for k equals 19 is 0.0583482320596
Average score for k equals 21 is 0.023170033898
Average score for k equals 23 is -0.0595299120651
Average score for k equals 25 is 0.0404855796837
Average score for k equals 27 is 0.0432195067493
Average score for k equals 29 is -0.0223238668632
Average score for k equals 31 is 0.0295702696463
Average score for k equals 33 is -0.0353385274806
Average score for k equals 35 is -0.0320955750387
Average score for k equals 37 is -0.0258361596227
Average score for k equals 39 is -0.00501551074733
Average score for k equals 41 is 0.0294567210567
Average score for k e

## Impriving data for modeling

From here the aim is to try and improve the model by selecting stores with more historical monthly data.

This is attempted by filtering out the stores that have less than 12 months of data, irrelevant of when the sales were recorded.

The average number of periods in the full data set (for which there are surrounding information) is 9.

In [633]:
print("The mean number of periods for the dataset is: {0}".format(str(reduce(lambda x, y: x + y, result['months_of_data_count']) / len(result['months_of_data_count']))))

# filter out stores with less than 12 periods
trimmed_result_df = result.loc[result['months_of_data_count'] > 11]

amenities_array = raw[0]['surroundings'].keys()

column_names_amenities = copy.deepcopy(amenities_array)
column_names_amenities = ['store_id'] + column_names_amenities

full_feature_amenities_df = pd.DataFrame(columns = column_names_amenities, index=[0])

for _, surroundings_obj in enumerate(raw):
    
    amenities_feature_dict = {}
    store_id = surroundings_obj['store_code']
    amenities_feature_dict['store_id'] = store_id
    
    for _, key in enumerate(amenities_array):
        amenities_feature_dict[key] = len(surroundings_obj['surroundings'][key])  
        
    feature_amenities_row = pd.DataFrame(data = amenities_feature_dict, columns = column_names_amenities, index=[0])
    
    full_feature_amenities_df = pd.concat([full_feature_amenities_df, feature_amenities_row])

full_feature_amenities_df = full_feature_amenities_df[1:]
full_feature_amenities_df.reset_index(drop=True, inplace=True)

# select the store ids that are only in the surrounding dataset
ops_df = trimmed_result_df.loc[trimmed_result_df['store_id'].isin(full_feature_amenities_df['store_id'])]

# join on store_id key to append the total_products_sold
# merged_df = pd.merge(full_feature_amenities_df, ops_df[['store_id', 'total_products_sold']], on='store_id')
merged_df = pd.merge(full_feature_amenities_df, ops_df[['store_id', 'mean']], on='store_id')

# make sure no NA rows are present (same dataset with or without the drop, but it's good practice)
merged_df.dropna()

The mean number of periods for the dataset is: 9


,store_id,subway_station,department_store,embassy,beauty_salon,police,courthouse,cemetery,pharmacy,local_government_office,...,zoo,train_station,jewelry_store,laundry,insurance_agency,plumber,pet_store,bakery,travel_agency,mean
0,10055,0,0,0,4,0,0,0,3,0,...,0,0,0,1,2,0,0,3,1,2252.00
1,10079,0,1,0,4,0,1,0,3,1,...,0,0,4,2,2,0,0,3,2,18014.00
2,10441,0,1,0,16,0,0,0,5,6,...,0,1,6,2,3,0,1,6,9,11470.00
3,10548,0,0,0,7,0,0,0,3,1,...,0,0,3,0,3,0,0,5,3,6406.88
4,10871,1,1,0,2,0,0,0,4,0,...,0,0,0,1,1,0,0,2,1,9022.00
5,10928,0,0,0,0,0,0,0,1,0,...,0,1,0,0,2,0,0,2,0,27388.42
6,11028,0,1,0,23,0,0,0,8,7,...,0,0,14,3,8,0,0,4,12,31708.24
7,11028,0,1,0,25,0,0,0,8,7,...,0,0,14,3,10,0,0,5,12,31708.24
8,11564,0,4,3,34,0,0,0,9,1,...,0,0,19,0,7,0,0,9,16,18220.00
9,11570,1,4,3,33,0,0,0,11,2,...,0,1,18,0,8,0,0,6,20,7854.00


## Fitting the model and checking results

In [634]:
# creating odd list of K for KNN
neigh_list = list(range(5,50))

# subsetting just the odd ones
neighbors = filter(lambda x: x % 2 != 0, neigh_list)

for _, weight_type in enumerate(['uniform', 'distance']):
    
    print('\n #### Starting with {} weight #### \n'.format(weight_type))
    
    
    for kn in neighbors:

        cross_validation_array = []

        for cv_iteration in range(0, 10):

            # sample random index numbers for splitting the dataset. Will go for 85% training and 15% testing.
            msk = np.random.rand(len(merged_df)) <= 0.8

            train_df = merged_df[msk]

            test_df = merged_df[~msk]

            training_fit_set_columns = [col for col in train_df.columns if col not in ['store_id', 'mean']]
            training_fit_df = normalize_df(train_df[training_fit_set_columns])
            training_target_df = train_df['mean']

            test_fit_set_columns = [col for col in test_df.columns if col not in ['store_id', 'mean']]
            test_fit_df = normalize_df(test_df[test_fit_set_columns])
            test_target_df = test_df['mean']

            # fit model 
            neigh = KNeighborsRegressor(n_neighbors=kn, weights=weight_type)
            neigh.fit(training_fit_df, training_target_df) 
            neigh.predict(test_fit_df)

            cross_validation_array.append(neigh.score(test_fit_df, test_target_df))


        avg_score = reduce(lambda x, y: x + y, cross_validation_array) / len(cross_validation_array)

        print('Average score for k equals {0} is {1}'.format(kn, avg_score))


 #### Starting with uniform weight #### 

Average score for k equals 5 is -0.124385872636
Average score for k equals 7 is -0.880689475988
Average score for k equals 9 is -0.98025514951
Average score for k equals 11 is -0.214581600559
Average score for k equals 13 is 0.0445381946342
Average score for k equals 15 is -0.0731680079781
Average score for k equals 17 is -0.555506478091
Average score for k equals 19 is -0.175933727659
Average score for k equals 21 is -0.50362226427
Average score for k equals 23 is -0.0347896218253
Average score for k equals 25 is -0.0320273994911
Average score for k equals 27 is -0.0665850542877
Average score for k equals 29 is -0.171834917999
Average score for k equals 31 is -0.0875260040147
Average score for k equals 33 is -0.0593168476046
Average score for k equals 35 is -0.434162040461
Average score for k equals 37 is -0.157082375382
Average score for k equals 39 is -0.0851629371799
Average score for k equals 41 is -0.126894259696
Average score for k equa

In [629]:
## next steps:
# Inspect the current dataset to see if there are any missing values in the total Numbers <> CHECK!
# Create test and training set + validation <> CHECK!
# Normalize the values per column (variable) between (0; 1). <> CHECK!
# Scale the Total sales by number of months taken into account and observe the difference to the model <> trimmed data with insufficient number of months

# if time allows (it's before 10:30) then inspect the other feature vectors. Include other feature vectors & vars (see how) and include map if possible.
## talk about it in the presentation only..

## extra option: make regression based on differences to cosine similarity to a benchmark, count of months and map area.
